
# Random Forest Algorithm
### Tag: Fashion MNIST, Random Forest without Scikit-learn, Image Recognition, multi-classification

## 0. Prepare Environment

In [1]:
!pip install -U pip
!pip install -U h5py numpy

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## 1. Load Data

In [2]:
import h5py
import numpy as np
with h5py.File('./data/train/images_training.h5', 'r') as H:
    data_train = np.copy(H['datatrain'])
with h5py.File('./data/train/labels_training.h5', 'r') as H:
    label_train = np.copy(H['labeltrain'])
with h5py.File('./data/test/images_testing.h5', 'r') as H:
    data_test = np.copy(H['datatest'])
with h5py.File('./data/test/labels_testing_2000.h5', 'r') as H:
    label_test = np.copy(H['labeltest'])

## 1.2 Keep Top 2000 Rows of Test Data

In [3]:
data_test = data_test[:2000]

## 2. Prepare Data Structure

## 2.1 Node

In [4]:
class node:
    def __init__(self, depth=0, value=None):
        self.depth = depth  # not necessary but can simplify bulid code
        self.left_child = None
        self.right_child = None
        self.value = value
        # If the data type of value is float -> it is a leaf node, the value is the probability
        # If the data type of value is int  -> it is a branch node, the value is the column name     

## 2.2 Decision Tree (Binary)

In [5]:
from copy import deepcopy
# Binary tree
class DecisionTree:
    def __init__(
        self,
        cls = 0,
        max_depth = 12,
        min_pct = 0.05,
        threshold = 0.32,
        worst_gini = 0.95
    ):
        self.root = node(depth=0)
        self.cls = cls
        self.max_depth = max_depth
        self.min_pct = min_pct
        self.threshold = threshold
        self.worst_gini = worst_gini
        self.data = None
        self.label = None


    def build(self, point, column, row):
        gini_list = [{'gini':1},]
        row_len = len(row)
        min_set_num = round(self.min_pct * row_len) + 1 # Ensure > 0
        # Scan each pixel in order to find out whether the pixel can effectively divide the data
        for i in column:
            condition = self.data[row,i] < self.threshold
            condition_true = sum(condition)
            condition_false = row_len - condition_true
            # If the data set cannot be divided into two parts,
            # The condition is useless
            if condition_true > min_set_num and condition_false > min_set_num:
                t_t = sum(condition & (self.label[row]==self.cls))
                f_t = sum(~condition & (self.label[row]==self.cls))
                p_tt = t_t / condition_true
                p_ft = f_t / condition_false
                gini = (1 - p_tt**2 - ((condition_true - t_t) / condition_true)**2)*(condition_true / row_len)\
                    + (1 - p_ft**2 - ((condition_false - f_t) / condition_false)**2)*(condition_false / row_len)
                gini_list.append({'gini':gini, 'column':i, 'condition':condition})
        best_res = min(gini_list, key=lambda x:x.get('gini'))
        if best_res.get('gini') < self.worst_gini and point.depth < self.max_depth:
            best_column = int(best_res.get('column'))
            point.value = best_column
            point.left_child = node(depth=point.depth + 1)
            point.right_child = node(depth=point.depth + 1)
            
            condition = best_res.get('condition')
            new_col = deepcopy(column)
            new_col.remove(best_column)
            self.build(point.left_child, new_col, row[condition])
            self.build(point.right_child, new_col, row[~condition])
        else:
            point.value = float(sum(self.label[row]==self.cls)/row_len)
            
            
    def fit(self, data, label):
        self.data = data
        self.label = label
        self.build(self.root, list(range(data.shape[1])), np.arange(data.shape[0]))

## 2.3 Random Forest(N v 1)

In [6]:
# N v 1: Each specific tree is only responsible for judging whether the data belongs to a specific classification
class RandomForest:
    def __init__(
        self,
        max_depth = 12,
        min_pct = 0.05,
        threshold = 0.3,
        worst_gini = 0.95
    ):
        self.root_list = [
            DecisionTree(
                cls = i,
                max_depth = max_depth,
                min_pct = min_pct,
                threshold = threshold,
                worst_gini = worst_gini
            )       for i in range(10)
        ]


    def fit(self, data_train, label_train):
        for tree_index, decision_tree in enumerate(self.root_list):
            print("start to build", tree_index, "decision tree")
            decision_tree.fit(data_train, label_train)

    def predict(self, data_test):
        predict = []
        for i in data_test:
            pro_list = []
            for dt in self.root_list:
                point = dt.root
                while 1:
                    if isinstance(point.value, float):
                        pro_list.append({'class':dt.cls, 'probability':point.value})
                        break
                    elif isinstance(point.value, int):
                        if i[point.value] < dt.threshold:
                            point = point.left_child
                        else:
                            point = point.right_child
                    else:
                        print('error:', str(type(point.value)))
            predict.append(max(pro_list,key=lambda x:x['probability'])['class'])
        return np.array(predict)

# 3 Train and predict

In [7]:
rf = RandomForest()
rf.fit(data_train, label_train)
predict = rf.predict(data_test[:2000])
print('Accuracy: ', sum(predict == label_test)/2000)

start to build 0 decision tree
start to build 1 decision tree
start to build 2 decision tree
start to build 3 decision tree
start to build 4 decision tree
start to build 5 decision tree
start to build 6 decision tree
start to build 7 decision tree
start to build 8 decision tree
start to build 9 decision tree
Accuracy:  0.755
